In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models.openai import ChatOpenAI
from langchain.schema.runnable import ConfigurableField
from operator import itemgetter
from langchain.schema.runnable import RunnableLambda
from langchain.schema import StrOutputParser

chat_model = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)
prompt = ChatPromptTemplate.from_template(
    "Tell me a joke about {input}"
).configurable_alternatives(
    ConfigurableField(id="prompt"),
    default_key="GPT3",
    GPT4=ChatPromptTemplate.from_template("Write a short poem about {input}"),
)
chain_t = prompt | chat_model | StrOutputParser()

def update_config(input):
    return chain_t.with_config(configurable={'prompt': input['_prompt']})

chain_with_config = lambda input: chain_t.with_config(configurable={'prompt': input['_prompt']}).invoke(input)



In [ ]:
chain_with_config({"input": "beer", "_prompt": "GPT4"})

In [ ]:
c = RunnableLambda(update_config)

In [ ]:
c.invoke({"input": "beer", "_prompt": "GPT4"})

In [ ]:
from setup import toolset, retriever, prompt_chat
from operator import itemgetter
import langchain
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.cache import InMemoryCache
from langchain.chat_models import ChatOpenAI, ChatCohere
from langchain.schema import StrOutputParser
from langchain.memory import ConversationBufferMemory
from langchain.schema.prompt import PromptValue
from langchain.schema.runnable import ConfigurableField, RunnableLambda, RunnablePassthrough
from langchain.tools.render import render_text_description
from langchain.prompts.chat import ChatPromptValue
from langchain.schema.runnable.base import RunnableBinding

# print 
def pp(p):
    print(p)
    print()
    return p

def tt(t):
    print(type(t))
    print()
    return t

# Cache
langchain.llm_cache = InMemoryCache()
# Memory
memory = ConversationBufferMemory(memory_key="history", return_messages=True)
# ChatModel & ConfigurableField
chat_model = ChatOpenAI(model="gpt-3.5-turbo-1106", 
                        temperature=0.9).configurable_alternatives(
                                            ConfigurableField(id="chatmodel"),
                                            default_key="GPT3",
                                            GPT4=ChatOpenAI(model="gpt-4-1106-preview"),
                                            Cohere=ChatCohere())

def config_chat(input):
    return chat_model.with_config(configurable={'chatmodel': input['_chatmodel']})

llm_with_stop = chat_model.bind(stop=["\nObservation"])

# Chain
chain = (
    RunnablePassthrough.assign(
        input=itemgetter("input"),
        context=RunnableLambda(lambda x: retriever.get_relevant_documents(x["input"])),
        chat_history=RunnableLambda(lambda x: memory.load_memory_variables(x["input"])["history"]),
        tools=RunnableLambda(lambda x: render_text_description(toolset)),
        tool_names=RunnableLambda(lambda x: ", ".join([t.name for t in toolset])),
        agent_scratchpad=RunnableLambda(lambda x: format_log_to_str(x["intermediate_steps"])),
    )
    | prompt_chat | RunnableLambda(pp) | RunnableLambda(tt)
    | config_chat | RunnableLambda(pp) | RunnableLambda(tt)
    | llm_with_stop | RunnableLambda(pp) | RunnableLambda(tt)
    | ReActSingleInputOutputParser()
)
# AgentExecutor with ConfigurableField
agent_executor = AgentExecutor(agent=chain, tools=toolset, 
                                verbose=True, memory=memory,                                 
                                max_execution_time=60, max_iterations=5,
                                handle_parsing_errors=True, early_stopping_method="generate")

c = RunnableLambda(tt) | RunnableLambda(pp) | agent_executor | RunnableLambda(pp)

c.invoke({"input": "beer", "_chatmodel": "GPT3"})


In [ ]:
chat_with_config = lambda input: RunnableLambda(llm_with_stop.with_config(configurable={'chatmodel': input['_chatmodel']}))


In [2]:
from langchain.chat_models.openai import ChatOpenAI
chat_model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0.3, streaming=True)

In [5]:
input ="""
Correctly order the execution of non-interactive zero-knowledge proofs with Zorc from A to H. Zocrates is a tool that makes it easy to implement non-interactive zero-knowledge proofs
(a) Computes a witness for the compiled program
(b) Generate a Solidity contract for Verifier Smart Contract
(c) Verify transaction
(d) Create Sudoku Zokrates Verification Circuit
(e) Creates a proving key and a verification key using “toxic waste”
(f) Compile Verification Circuit
(g) Deploy Smart Contract to Blockchain
(h) Generate Proof
"""

In [6]:
ans = chat_model.invoke(input)
print(ans.content)

To correctly order the execution of non-interactive zero-knowledge proofs using Zokrates, follow these steps:

1. (d) Create Sudoku Zokrates Verification Circuit - First, you need to create the verification circuit for the specific computation you want to prove, in this case, a Sudoku puzzle.

2. (f) Compile Verification Circuit - After creating the circuit, you compile it to generate the arithmetic representation that Zokrates can work with.

3. (e) Creates a proving key and a verification key using “toxic waste” - With the compiled circuit, you then use the Zokrates setup process to generate a proving key (for creating proofs) and a verification key (for verifying proofs). The "toxic waste" refers to the randomness used in this process that must be securely discarded to ensure the security of the system.

4. (a) Computes a witness for the compiled program - Once you have the proving key, you compute a witness for the specific instance of the problem you are proving (e.g., a solution 